<a href="https://colab.research.google.com/github/RajAakash/TransferLearningPytorch/blob/main/DataParallelPytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a large neural network that doesn't fit in a single GPU
class LargeModel(nn.Module):
    def __init__(self):
        super(LargeModel, self).__init__()
        self.layer1 = nn.Linear(10000, 5000)
        self.layer2 = nn.Linear(5000, 1000)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        return x

model = LargeModel()

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

model = model.to(device)
if device.type == "cuda" and torch.cuda.device_count() > 1:
    print('I have multiple GPU\'s and run in parallel')
    model = nn.DataParallel(model)

data = torch.randn(64, 10000).to(device)
target = torch.randn(64, 1000).to(device)
dataset = torch.utils.data.TensorDataset(data, target)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=16)

criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

for epoch in range(10):
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

print("Training completed!")

Training completed!


In [2]:
import torch

# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    print(f"Number of available GPUs: {device_count}")

    for i in range(device_count):
        gpu = torch.device(f"cuda:{i}")
        print(f"GPU {i} - {torch.cuda.get_device_name(gpu)}")
        print(f"Utilization: {torch.cuda.max_memory_allocated(gpu) / 1024**2:.2f} MB")

Number of available GPUs: 1
GPU 0 - Tesla T4
Utilization: 442.71 MB
